In [4]:
import torch.utils
import torch.utils.data
from LSTM.datapreprocessing import create_data

import csv
import os
import pandas as pd
import numpy as np
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F


import torch
import torch.nn as nn

In [5]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, future_timesegments):
        super(SimpleRNN, self).__init__()
        self.future_timesegments = future_timesegments
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.LSTM_cell = nn.LSTM(input_size, hidden_size,  num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        # self.fc2 = nn.Linear(hidden_size, 50)
        self.fc3 = nn.Linear(hidden_size,output_size)


    def forward(self, x, hidden):
        # Forward pass through the RNN layer
        out, hidden = self.LSTM_cell(x, hidden)
        # Reshape the output to fit into the fully connected layer
        # out = out.contiguous().view(-1, self.hidden_size) many to one
        out = out[:, -4:, :] # --> Last time step 
        # out = F.relu(self.fc1(out))
        out = F.relu(self.fc1(out))
        # out = F.sigmoid(self.fc2(out))
        out = self.fc3(out)
        # print(out.shape)
        return out, hidden

    def init_hidden(self, x):
        # Initialize hidden state with zeros
        return (torch.zeros(self.num_layers, x.size(0), self.hidden_size),torch.zeros(self.num_layers, x.size(0), self.hidden_size))

In [11]:
input = torch.tensor([[[ 0.0000e+00,  0.0000e+00],
        [-1.6442e+00, -1.8951e-02],
        [-5.6885e+00, -5.9998e-02],
        [-9.8572e+00,  1.2701e-01],
        [-1.4382e+01,  1.8763e+00]]])
input2 = torch.tensor([[[ 0.0000e+00,  0.0000e+00],
        [0,0],
        [0,0],
        [0,  0],
        [0,  0]]])
# Define input, hidden, and output sizes
input_size = 2  # Size of input vectors (x and y coordinates)
hidden_size = 500  # Size of hidden state (hyperparameter)
output_size = 2  # Size of output vectors (x and y coordinates)
num_layers = 2# amount of layers (hyperparameter)
future_timesegments = 4
model_name = 'LSTM_PAST5_FUTURE4_H500_L2_RELU.pt'
LSTM_model = SimpleRNN(input_size, hidden_size, output_size, num_layers, future_timesegments)
LSTM_model = torch.load('models/' + model_name)
hidden = LSTM_model.init_hidden(input)
future_pred, _ = LSTM_model(input2,hidden)
print(input2)
print(future_pred)



tensor([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]])
tensor([[[-0.1943, -0.0167],
         [-0.2057, -0.0056],
         [-0.2193, -0.0042],
         [-0.2408, -0.0019]]], grad_fn=<ViewBackward0>)
